In [1]:
%pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 8.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 8.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 9.1 MB/s eta 0:00:00:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.5/502.5 kB 6.0 MB/s eta 0:00:0000:01:00:01
Note: you may need to restart the kernel to use updated packages.


## CSVを以下の形式に変換
Year,Country,TotalPopulation  
1964,Afghanistan,9744.78  
1965,Afghanistan,9956.32  
1966,Afghanistan,10174.84  
1967,Afghanistan,10399.93  

....  

2019,Afghanistan,38041.75  
2020,Afghanistan,38928.35  
1964,Albania,1843.63  
1965,Albania,1896.17  

....    
....  

2019,Zimbabwe,14645.47  
2020,Zimbabwe,14862.92  


In [6]:
import pandas as pd
import os

# ファイル名を指定
#filename = 'TotalPopulation.csv'
name = 'IndustrialWaterWithdrawal.csv'
name = 'AgriculturalWaterWithdrawal.csv'
name = 'MunicipalWaterWithdrawal.csv'
name = 'TotalPopulation.csv'
name = 'TotalWaterWithdrawal.csv'
name = 'TotalWaterWithdrawalPerCapita.csv'
name = 'GDP.csv'
name = 'TotalPopulationWithAccessToSafeDrinking-water.csv'

filename = 'rawdata/' + name

# ファイル名から拡張子を除いた部分を取得
value_name = os.path.splitext(name)[0]

# CSVファイルを読み込む
df = pd.read_csv(filename) 

# 列名を整理する（年の列名から引用符を削除）
df.columns = df.columns.str.replace('"', '')

# 年の列を縦に並べるためにmelt関数を使用
df_melt = df.melt(id_vars=['Country'], var_name='Year')

# 'value'列を数値に変換
df_melt['value'] = pd.to_numeric(df_melt['value'], errors='coerce')

# 'Year'列の余分な空白を削除
df_melt['Year'] = df_melt['Year'].str.strip()

# 'Country'と'Year'をindexに設定し、'value'を列に設定してピボット
# 複数の値が存在する場合は最初に見つかった値を使用
df_pivot = df_melt.pivot_table(index='Year', columns='Country', values='value', aggfunc='first')


# df_pivotという名前のDataFrameがあると仮定
# そのDataFrameをリセットインデックス
df_reset = df_pivot.reset_index()

# DataFrameを長い形式に変換
df_melted = df_reset.melt(id_vars='Year', var_name='Country', value_name=value_name)

# 結果をCSVファイルに出力
df_melted.to_csv(f'1st_formatted_data/{value_name}_1stformatted.csv', index=False)

## 2つ以上のCSVを以下のように統合するコード


Year,Country,TotalPopulation  
1964,Afghanista,9744.78  
1964,Albania,1843.63  
...  


Year,Country,AgriculturalWaterWithdrawal  
1965,Afghanistan,  
1966,Afghanistan,  
...  


Year,Country,IndustrialWaterWithdrawal  
1965,Afghanistan,  
1966,Afghanistan,  
...


上記のような3つのCSVを、下記のように統合する  
  
Year,Country,TotalPopulation,AgriculturalWaterWithdrawal,IndustrialWaterWithdrawal   
1964,Afghanista,9744.78,,  
1964,Albania,1843.63,,  
...  



In [7]:
import pandas as pd

# 各CSVファイルを読み込む
df_population = pd.read_csv('1st_formatted_data/TotalPopulation_1stformatted.csv')
df_agri_water = pd.read_csv('1st_formatted_data/AgriculturalWaterWithdrawal_1stformatted.csv')
df_indus_water = pd.read_csv('1st_formatted_data/IndustrialWaterWithdrawal_1stformatted.csv')
df_muni_water = pd.read_csv('1st_formatted_data/MunicipalWaterWithdrawal_1stformatted.csv')
df_total_water = pd.read_csv('1st_formatted_data/TotalWaterWithdrawal_1stformatted.csv')
df_total_water_per_capita = pd.read_csv('1st_formatted_data/TotalWaterWithdrawalPerCapita_1stformatted.csv')
df_GDP = pd.read_csv('1st_formatted_data/GDP_1stformatted.csv')
df_total_population_with_access_to_safe_drinking_water = pd.read_csv('1st_formatted_data/TotalPopulationWithAccessToSafeDrinking-water_1stformatted.csv')


# 'Year'と'Country'を基にして3つのデータフレームを結合
df_merged = pd.merge(df_population, df_agri_water, on=['Year', 'Country'], how='outer')
df_merged = pd.merge(df_merged, df_indus_water, on=['Year', 'Country'], how='outer')
df_merged = pd.merge(df_merged, df_muni_water, on=['Year', 'Country'], how='outer')
df_merged = pd.merge(df_merged, df_total_water, on=['Year', 'Country'], how='outer')
df_merged = pd.merge(df_merged, df_total_water_per_capita, on=['Year', 'Country'], how='outer')
df_merged = pd.merge(df_merged, df_GDP, on=['Year', 'Country'], how='outer')
df_merged = pd.merge(df_merged, df_total_population_with_access_to_safe_drinking_water, on=['Year', 'Country'], how='outer')


# 結果をCSVファイルに出力
df_merged.to_csv('MergedData.csv', index=False)

In [4]:
import pandas as pd
a = pd.read_csv('MergedData.csv')

In [5]:
a

,Year,Country,TotalPopulation,AgriculturalWaterWithdrawal,IndustrialWaterWithdrawal,MunicipalWaterWithdrawal,TotalWaterWithdrawal,TotalWaterWithdrawalPerCapita,GDP,TotalPopulationWithAccessToSafeDrinkingWater
0,1964,Afghanistan,9744.78,NaN,NaN,NaN,NaN,NaN,8.000000e+08,NaN
1,1965,Afghanistan,9956.32,NaN,NaN,NaN,NaN,NaN,1.006667e+09,NaN
2,1966,Afghanistan,10174.84,NaN,NaN,NaN,NaN,NaN,1.400000e+09,NaN
3,1967,Afghanistan,10399.93,NaN,NaN,NaN,NaN,NaN,1.673333e+09,NaN
4,1968,Afghanistan,10637.06,NaN,NaN,NaN,NaN,NaN,1.373333e+09,NaN
...,...,...,...,...,...,...,...,...,...,...
11395,2016,Zimbabwe,14030.33,2.72,0.08,0.60,3.40,242.45,2.054868e+10,76.9
11396,2017,Zimbabwe,14236.59,2.77,0.08,0.49,3.34,234.54,2.204090e+10,76.9
11397,2018,Zimbabwe,14438.80,3.04,0.08,0.65,3.77,261.20,2.431156e+10,76.9
11398,2019,Zimbabwe,14645.47,3.04,0.08,0.65,3.77,257.51,2.193508e+10,76.9


In [ ]:
a['TotalPopulationWithoutAccessToSafeDrinkingWater'] = a['TotalPopulation']*(100 - a['TotalPopulationWithAccessToSafeDrinkingWater'])

In [8]:
a['TotalPopulationWithoutAccessToSafeDrinkingWater'] = a.apply(
    lambda row: row['TotalPopulation'] * (1.0 - 0.01*row['TotalPopulationWithAccessToSafeDrinkingWater'])
    if not pd.isnull(row['TotalPopulation']) and not pd.isnull(row['TotalPopulationWithAccessToSafeDrinkingWater'])
    else None, axis=1
)

In [9]:
a

,Year,Country,TotalPopulation,AgriculturalWaterWithdrawal,IndustrialWaterWithdrawal,MunicipalWaterWithdrawal,TotalWaterWithdrawal,TotalWaterWithdrawalPerCapita,GDP,TotalPopulationWithAccessToSafeDrinkingWater,TotalPopulationWithoutAccessToSafeDrinkingWater
0,1964,Afghanistan,9744.78,NaN,NaN,NaN,NaN,NaN,8.000000e+08,NaN,NaN
1,1965,Afghanistan,9956.32,NaN,NaN,NaN,NaN,NaN,1.006667e+09,NaN,NaN
2,1966,Afghanistan,10174.84,NaN,NaN,NaN,NaN,NaN,1.400000e+09,NaN,NaN
3,1967,Afghanistan,10399.93,NaN,NaN,NaN,NaN,NaN,1.673333e+09,NaN,NaN
4,1968,Afghanistan,10637.06,NaN,NaN,NaN,NaN,NaN,1.373333e+09,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
11395,2016,Zimbabwe,14030.33,2.72,0.08,0.60,3.40,242.45,2.054868e+10,76.9,3241.00623
11396,2017,Zimbabwe,14236.59,2.77,0.08,0.49,3.34,234.54,2.204090e+10,76.9,3288.65229
11397,2018,Zimbabwe,14438.80,3.04,0.08,0.65,3.77,261.20,2.431156e+10,76.9,3335.36280
11398,2019,Zimbabwe,14645.47,3.04,0.08,0.65,3.77,257.51,2.193508e+10,76.9,3383.10357


In [10]:
a.to_csv('MergedData.csv', index=False)